In [1]:
### Options:

# Screen recording settings
screen_record = False  # Change to True for screen recording mode
num_opponents = 4
conf_value = 0.25
iou_value = 0.5

In [2]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import cv2
from ultralytics import YOLO
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import time
import pyautogui
import tkinter as tk

In [3]:
model_8_last = YOLO("yolov8n_last.pt")
model_8_best = YOLO("yolov8n_best.pt")
model_11n_last = YOLO("yolo11n_last.pt")
model_11n_best = YOLO("yolo11n_best.pt")
model_11m_last = YOLO("yolo11m_last.pt")
model_11m_best = YOLO("yolo11m_best.pt")
model_11m_42_last = YOLO("yolo11m_42_last.pt")
model_11m_42_best = YOLO("yolo11m_42_best.pt")
list_models = [
    # model_8_last,
    # model_8_best,
    # model_11n_last,
    # model_11n_best,
    # model_11m_last,
    # model_11m_best,
    # model_11m_42_last,
    model_11m_42_best
]


class_names = ['10C', '10D', '10H', '10S', '2C', '2D', '2H', '2S', '3C', '3D', '3H', '3S', '4C', '4D', '4H', '4S', '5C', '5D', '5H', '5S', '6C', '6D', '6H', '6S', '7C', '7D', '7H', '7S', '8C', '8D', '8H', '8S', '9C', '9D', '9H', '9S', 'AC', 'AD', 'AH', 'AS', 'JC', 'JD', 'JH', 'JS', 'KC', 'KD', 'KH', 'KS', 'QC', 'QD', 'QH', 'QS']


In [4]:
class_names2=[]
for i,v in enumerate(class_names):
    if i%4==0:
        out = v[:-1]+"♣"
    if i%4==1:
        out = v[:-1]+"♦"
    if i%4==2:
        out = v[:-1]+"♥"
    if i%4==3:
        out = v[:-1]+"♠"
    class_names2.append(out)
    # print(out)

print(class_names)
print(class_names2)

['10C', '10D', '10H', '10S', '2C', '2D', '2H', '2S', '3C', '3D', '3H', '3S', '4C', '4D', '4H', '4S', '5C', '5D', '5H', '5S', '6C', '6D', '6H', '6S', '7C', '7D', '7H', '7S', '8C', '8D', '8H', '8S', '9C', '9D', '9H', '9S', 'AC', 'AD', 'AH', 'AS', 'JC', 'JD', 'JH', 'JS', 'KC', 'KD', 'KH', 'KS', 'QC', 'QD', 'QH', 'QS']
['10♣', '10♦', '10♥', '10♠', '2♣', '2♦', '2♥', '2♠', '3♣', '3♦', '3♥', '3♠', '4♣', '4♦', '4♥', '4♠', '5♣', '5♦', '5♥', '5♠', '6♣', '6♦', '6♥', '6♠', '7♣', '7♦', '7♥', '7♠', '8♣', '8♦', '8♥', '8♠', '9♣', '9♦', '9♥', '9♠', 'A♣', 'A♦', 'A♥', 'A♠', 'J♣', 'J♦', 'J♥', 'J♠', 'K♣', 'K♦', 'K♥', 'K♠', 'Q♣', 'Q♦', 'Q♥', 'Q♠']


In [5]:
# List to store corrected bounding boxes and category labels
def select_screen_region():
    """
    Opens a transparent overlay to allow the user to click and drag to define a recording region.
    Returns the coordinates (xmin, ymin, xmax, ymax) of the selected region.
    """
    time.sleep(3)
    coords = {}

    def on_press(event):
        coords["x1"], coords["y1"] = event.x, event.y
        # Create a rectangle on canvas to show the selection
        canvas.delete("rect")
        canvas.create_rectangle(coords["x1"], coords["y1"], event.x, event.y, outline="red", width=2, tags="rect")

    def on_drag(event):
        # Update the rectangle while dragging
        canvas.delete("rect")
        canvas.create_rectangle(coords["x1"], coords["y1"], event.x, event.y, outline="red", width=2, tags="rect")

    def on_release(event):
        coords["x2"], coords["y2"] = event.x, event.y
        root.quit()

    # Create a transparent overlay
    root = tk.Tk()
    root.attributes('-fullscreen', True)
    root.attributes('-alpha', 0.2)  # Background is transparent, making it less intrusive
    root.configure(bg='black')

    canvas = tk.Canvas(root, cursor="cross")
    canvas.pack(fill="both", expand=True)

    canvas.bind("<ButtonPress-1>", on_press)
    canvas.bind("<B1-Motion>", on_drag)
    canvas.bind("<ButtonRelease-1>", on_release)

    root.mainloop()
    root.destroy()

    # Extract coordinates of the selected region
    x1, y1, x2, y2 = coords["x1"], coords["y1"], coords["x2"], coords["y2"]
    xmin, ymin = min(x1, x2), min(y1, y2)
    xmax, ymax = max(x1, x2), max(y1, y2)

    return xmin, ymin, xmax, ymax

if screen_record:
    xmin, ymin, xmax, ymax = select_screen_region() # Define the screen capture region
# Helper functions for detection

def resizer(frame,screen_record):
    if screen_record:
        return frame
    else:
        # resized_image = cv2.resize(frame,(640,640))
        return frame
    return resized_image

def calculate_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    intersection_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - intersection_area

    return intersection_area / union_area if union_area > 0 else 0

def filter_boxes_by_iou(boxes, iou_threshold=0.5):
    boxes = sorted(boxes, key=lambda x: x[4], reverse=True)
    filtered_boxes = []

    while boxes:
        best_box = boxes.pop(0)
        filtered_boxes.append(best_box)
        boxes = [
            box for box in boxes
            if calculate_iou(best_box[:4], box[:4]) < iou_threshold
        ]
    return filtered_boxes

def round_data(box):
    return [
        np.round(box[0], 0),
        np.round(box[1], 0),
        np.round(box[2], 0),
        np.round(box[3], 0),
        box[4],
        int(np.round(box[5], 0))
    ]

def get_results_given_model(frame, model_list, conf=0.25, iou_threshold=0.5):
    resized_frame = resizer(frame,screen_record)
    results_per_model = []

    for model in model_list:
        results = model.predict(source=resized_frame, conf=conf)
        raw_boxes = results[0].boxes.data.tolist()
        rounded_boxes = [round_data(box) for box in raw_boxes]
        filtered_boxes = filter_boxes_by_iou(rounded_boxes, iou_threshold=iou_threshold)
        results_per_model.append(filtered_boxes)

    return results_per_model

# Selenium automation
def initialize_browser(url):
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--window-size=600,2300")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])

    service = Service()
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(url)
    driver.execute_script("document.body.style.zoom='100%'")
    return driver

def select_card(driver, dropdown_id, card_value):
    dropdown = driver.find_element(By.ID, dropdown_id)
    ActionChains(driver).move_to_element(dropdown).click().perform()
    options_container = driver.find_element(By.CLASS_NAME, "Select-menu")
    option = options_container.find_element(By.XPATH, f".//div[contains(text(), '{card_value}')]")
    option.click()

def select_hand(driver, hand_cards, class_names2):
    for card, dropdown_id in zip(hand_cards, ["react-select-2--value-item", "react-select-3--value-item"]):
        select_card(driver, dropdown_id, class_names2[card[-1]])

def select_community(driver, community_cards, class_names2):
    for card, dropdown_id in zip(community_cards, ["react-select-4--value-item", "react-select-5--value-item", 
                                                   "react-select-6--value-item", "react-select-7--value-item",
                                                   "react-select-8--value-item"]):
        select_card(driver, dropdown_id, class_names2[card[-1]])

# Main camera and calculator interaction
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Cannot open camera")
    exit()

frozen_frame = None
frozen_bbox = None
is_frozen = False
results = None
webpage_url = "https://www.omnicalculator.com/other/poker-odds"
driver = None

while True:
    if not is_frozen:
        if screen_record:
            # Capture the screen region
            screen = pyautogui.screenshot(region=(xmin, ymin, xmax-xmin , ymax-ymin))
            frame = cv2.cvtColor(np.array(screen), cv2.COLOR_RGB2BGR)
        else:
            # Capture from the external camera
            ret, frame = cap.read()
            frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
            if not ret:
                print("Failed to grab frame. Exiting...")
                break
    else:
        frame = frozen_frame.copy()


    if not is_frozen or (is_frozen and frozen_frame is not None):
        if not is_frozen:
            results = get_results_given_model(frame, list_models, conf=conf_value, iou_threshold=iou_value)
        current_bbox = results if not is_frozen else frozen_bbox
        for boxes in current_bbox:
            for box in boxes:
                x_min, y_min, x_max, y_max, confidence, class_id = box
                cv2.rectangle(frame, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 0, 255), 2)
                cv2.putText(frame, f"{class_names[class_id]} ({confidence:.2f})", (int(x_min), int(y_min) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    cv2.imshow('YOLO Detection', frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord(' '):  # Spacebar toggles freeze and calculates odds
        if is_frozen:
            is_frozen = False
        else:
            is_frozen = True
            frozen_frame = frame.copy()
            frozen_bbox = results

            # Extract cards from results
            set_cards = set()
            set_values = set()

            for i in frozen_bbox[0]:
                value = i[-1]
                if value not in set_values:
                    set_values.add(value)
                    set_cards.add(tuple(i))

            list_cards = list(set_cards)
            sorted_list = sorted(list_cards, key=lambda x: x[0] + (3*x[1]), reverse=True)


            hand = [sorted_list.pop(0), sorted_list.pop(0)]
            community = sorted(sorted_list, key=lambda x: x[0], reverse=False)

            # Launch browser and interact with the calculator
            driver = initialize_browser(webpage_url)
            select_card(driver, "react-select-9--value-item", str(num_opponents))  # Number of opponents
            select_hand(driver, hand, class_names2)
            select_community(driver, community, class_names2)
            


# Cleanup
if not screen_record:
    cap.release()
if driver:
    driver.quit()
cv2.destroyAllWindows()



0: 640x480 (no detections), 368.6ms
Speed: 4.0ms preprocess, 368.6ms inference, 87.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 305.6ms
Speed: 2.0ms preprocess, 305.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 303.6ms
Speed: 1.0ms preprocess, 303.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 323.4ms
Speed: 2.0ms preprocess, 323.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 302.6ms
Speed: 2.0ms preprocess, 302.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 300.7ms
Speed: 2.0ms preprocess, 300.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 318.0ms
Speed: 1.0ms preprocess, 318.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 309.6ms
Speed: 3.0ms prep